In [ ]:
%tensorflow_version 1.x
from __future__ import print_function

import numpy as np
import datetime
import tensorflow as tf

TensorFlow 1.x selected.


## Preparation

In [ ]:
%%time
log_device_placement = True
# Number of matrix multiplications to perform
n = 10

# Generate random large matrix
A = np.random.rand(10000, 10000).astype('float32')
B = np.random.rand(10000, 10000).astype('float32')

# Store results
c1 = []
c2 = []

CPU times: user 1.64 s, sys: 276 ms, total: 1.92 s
Wall time: 1.92 s


## Operations

In [ ]:
def matpow(M, n):
    if n < 1:
        return M
    else:
        return tf.matmul(M, matpow(M, n-1))

## CPU Experiment

In [ ]:
with tf.device('/cpu:0'):
    a = tf.placeholder(tf.float32, [10000, 10000])
    b = tf.placeholder(tf.float32, [10000, 10000])
    c1.append(matpow(a, n))
    c1.append(matpow(b, n))
    sum = tf.add_n(c1)

t1_1 = datetime.datetime.now()
with tf.Session(config=tf.ConfigProto(log_device_placement=log_device_placement)) as sess:
    sess.run(sum, {a:A, b:B})
t2_1 = datetime.datetime.now()
print("CPU computation time: " + str(t2_1-t1_1))

CPU Experiment (R5 2600) computation time: 0:03:27.089997

## Single GPU Experiment (1x 1080Ti)

In [ ]:
with tf.device('/gpu:0'):
    a = tf.placeholder(tf.float32, [10000, 10000])
    b = tf.placeholder(tf.float32, [10000, 10000])
    c1.append(matpow(a, n))
    c1.append(matpow(b, n))
    sum = tf.add_n(c1)

t1_2 = datetime.datetime.now()
with tf.Session(config=tf.ConfigProto(log_device_placement=log_device_placement)) as sess:
    sess.run(sum, {a:A, b:B})
t2_2 = datetime.datetime.now()
print("Single GPU computation time: " + str(t2_2-t1_2))

Single GPU Experiment (1x 1080Ti) computation time: 0:00:4.432052

## Multi GPU Experiment (2x GTX 1080 Ti)

In [ ]:
with tf.device('/gpu:0'):
    a = tf.placeholder(tf.float32, [10000, 10000])
    c1.append(matpow(a, n))
with tf.device('/gpu:1'):    
    b = tf.placeholder(tf.float32, [10000, 10000])
    c1.append(matpow(b, n))
with tf.device('/cpu:0'):
    sum = tf.add_n(c1)

t1_3 = datetime.datetime.now()
with tf.Session(config=tf.ConfigProto(log_device_placement=log_device_placement)) as sess:
    sess.run(sum, {a:A, b:B})
t2_3 = datetime.datetime.now()
print("Multiple GPU computation time: " + str(t2_3-t1_3))

Multi GPU Experiment (2x 1080Ti) computation time: 0:00:2.891781

## Mixed CPU-GPU with Multi GPU

In [ ]:
with tf.device('/gpu:0'):
    a = tf.placeholder(tf.float32, [10000, 10000])
    c1.append(matpow(a, n))
with tf.device('/gpu:1'):    
    b = tf.placeholder(tf.float32, [10000, 10000])
    c1.append(matpow(b, n))
with tf.device('/cpu:0'):
    sum = tf.add_n(c1)

t1_4 = datetime.datetime.now()
with tf.Session(config=tf.ConfigProto(log_device_placement=log_device_placement)) as sess:
    sess.run(sum, {a:A, b:B})
t2_4 = datetime.datetime.now()
print("Multiple GPU computation time: " + str(t2_4-t1_4))

Mixed CPU-GPU with Multi GPU (2x 1080Ti + R5 2600) computation time: 0:00:02.175499